# A full training

Install the Datasets, Evaluate, Transformers and Accelerate libraries to run this notebook.

In [1]:
!pip install -q datasets evaluate transformers[sentencepiece]
!pip install -q accelerate
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 30.5 MB/s et

To run the training on a TPU, you will need to uncomment and run the following line:

In [2]:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

If running this notebook in Colab, please ensure that your Hugging Face `HF_TOKEN` and your Weights & Biases `WANDB_API_KEY` are added to your Colab secrets.

Alternatively, please login to Hugging Face and Weights & Biases by running the following two cells.

In [3]:
# !huggingface-cli login

In [4]:
# !wandb login

In [5]:
import os
import random
import numpy as np
import torch

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [6]:
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

We shall load the "`mrpc`" subset (a.k.a. 'configuration') of the `"glue"` dataset.

From the <a href="https://huggingface.co/datasets/glue" target="_blank">dataset page</a>:

> GLUE, the General Language Understanding Evaluation benchmark (https://gluebenchmark.com/) is a collection of resources for training, evaluating, and analyzing natural language understanding systems.

`"mrpc"`:

> The Microsoft Research Paraphrase Corpus (Dolan & Brockett, 2005) is a corpus of sentence pairs automatically extracted from online news sources, with human annotations for whether the sentences in the pair are semantically equivalent.

From the <a href="https://huggingface.co/docs/datasets/v2.16.1/en/package_reference/loading_methods#datasets.load_dataset" target="_blank">documentation</a> of `load_dataset`, the first two parameters are:

- **path** (`str`) — Path or name of the dataset.

- **name** (`str`, *optional*) — Defining the name of the dataset configuration.

In [8]:
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [9]:
features = raw_datasets['train'].features
features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [10]:
id2label = {id: label for id, label in enumerate(features['label'].names)}
id2label

{0: 'not_equivalent', 1: 'equivalent'}

In [11]:
label2id = {label: id for id, label in id2label.items()}
label2id

{'not_equivalent': 0, 'equivalent': 1}

In [12]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.model_max_length

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

512

In [13]:
example = raw_datasets['train'][0]
example

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [14]:
encoding = tokenizer(example['sentence1'], example['sentence2'])
encoding

{'input_ids': [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer.decode(encoding['input_ids'])

'[CLS] amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence. [SEP] referring to him as only " the witness ", amrozi accused his brother of deliberately distorting his evidence. [SEP]'

In [16]:
tokenizer.decode(encoding['input_ids'], skip_special_tokens=True)

'amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence. referring to him as only " the witness ", amrozi accused his brother of deliberately distorting his evidence.'

Alt:

In [17]:
sentence_pair = example['sentence1'] + " [SEP] " + example['sentence2']
sentence_pair

'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence . [SEP] Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'

In [18]:
encoding = tokenizer(sentence_pair)
encoding

{'input_ids': [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

**Note:** The token type IDs are all zeros.

**Question:** Are the token type IDs actually used for sequence / sequence pair classification? Or are they just a relic of the next sentence prediction (NSP) auxilliary task of masked language modeling (i.e., BERT pre-training)?

In [19]:
tokenizer.decode(encoding['input_ids'])

'[CLS] amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence. [SEP] referring to him as only " the witness ", amrozi accused his brother of deliberately distorting his evidence. [SEP]'

In [20]:
def tokenize_function(example):
    return tokenizer(example['sentence1'], example['sentence2'], truncation=True, max_length=512)

In [21]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

**Note:** The tokenization was pretty straight forward (compared to, for example the tokenization approach in The Kaggle Book.)

In [22]:
example0 = tokenized_datasets['train'][0]
type(example0)

dict

In [23]:
for k, v in example0.items():
    print(f"{k}: {v}")

sentence1: Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
sentence2: Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .
label: 1
idx: 0
input_ids: [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102]
token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [24]:
id2label[example0['label']]

'equivalent'

In [25]:
len(example0['input_ids']), len(example0['token_type_ids']), len(example0['attention_mask'])

(50, 50, 50)

In [26]:
example1 = tokenized_datasets['train'][1]
type(example1)

dict

In [27]:
for k, v in example1.items():
    print(f"{k}: {v}")

sentence1: Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .
sentence2: Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .
label: 0
idx: 1
input_ids: [101, 9805, 3540, 11514, 2050, 3079, 11282, 2243, 1005, 1055, 2077, 4855, 1996, 4677, 2000, 3647, 4576, 1999, 2687, 2005, 1002, 1016, 1012, 1019, 4551, 1012, 102, 9805, 3540, 11514, 2050, 4149, 11282, 2243, 1005, 1055, 1999, 2786, 2005, 1002, 6353, 2509, 2454, 1998, 2853, 2009, 2000, 3647, 4576, 2005, 1002, 1015, 1012, 1022, 4551, 1999, 2687, 1012, 102]
token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [28]:
id2label[example1['label']]

'not_equivalent'

In [29]:
len(example1['input_ids']), len(example1['token_type_ids']), len(example1['attention_mask'])

(59, 59, 59)

In [30]:
tokenized_datasets = tokenized_datasets.remove_columns(['sentence1', 'sentence2', 'idx'])
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')
tokenized_datasets.set_format("torch")
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

**Note:**

1. When using the Trainer API, we don't have to rename `'label'` to `'labels'`. The Trainer API takes care of it.
2. `set_format` is the inplace version of `with_format`.

In [31]:
example0 = tokenized_datasets['train'][0]
type(example0)

dict

In [32]:
for k, v in example0.items():
    print(f"{k}: {v}")

labels: 1
input_ids: tensor([  101,  2572,  3217,  5831,  5496,  2010,  2567,  1010,  3183,  2002,
         2170,  1000,  1996,  7409,  1000,  1010,  1997,  9969,  4487, 23809,
         3436,  2010,  3350,  1012,   102,  7727,  2000,  2032,  2004,  2069,
         1000,  1996,  7409,  1000,  1010,  2572,  3217,  5831,  5496,  2010,
         2567,  1997,  9969,  4487, 23809,  3436,  2010,  3350,  1012,   102])
token_type_ids: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1])
attention_mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1])


In [33]:
example0['labels']

tensor(1)

In [34]:
id2label[example0['labels'].item()]

'equivalent'

In [35]:
example0['labels'].shape, example0['input_ids'].shape, example0['token_type_ids'].shape, example0['attention_mask'].shape

(torch.Size([]), torch.Size([50]), torch.Size([50]), torch.Size([50]))

In [36]:
example1 = tokenized_datasets['train'][1]
type(example1)

dict

In [37]:
for k, v in example1.items():
    print(f"{k}: {v}")

labels: 0
input_ids: tensor([  101,  9805,  3540, 11514,  2050,  3079, 11282,  2243,  1005,  1055,
         2077,  4855,  1996,  4677,  2000,  3647,  4576,  1999,  2687,  2005,
         1002,  1016,  1012,  1019,  4551,  1012,   102,  9805,  3540, 11514,
         2050,  4149, 11282,  2243,  1005,  1055,  1999,  2786,  2005,  1002,
         6353,  2509,  2454,  1998,  2853,  2009,  2000,  3647,  4576,  2005,
         1002,  1015,  1012,  1022,  4551,  1999,  2687,  1012,   102])
token_type_ids: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
attention_mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [38]:
example1['labels']

tensor(0)

In [39]:
id2label[example1['labels'].item()]

'not_equivalent'

In [40]:
example1['labels'].shape, example1['input_ids'].shape, example1['token_type_ids'].shape, example1['attention_mask'].shape

(torch.Size([]), torch.Size([59]), torch.Size([59]), torch.Size([59]))

In [41]:
tokenized_datasets['train'].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

`DataCollatorWithPadding` will allow us to use dynamic padding for our batches.

In [42]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

**Note:** The `Engine` class can be instantiated right after this step.

In [43]:
batch_size = 8

In [44]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets['train'],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets['validation'],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator
)

len(train_dataloader), len(eval_dataloader)

(459, 51)

There are 459 batches in `train_dataloader` and 51 batches in `eval_dataloader`.

In [45]:
for batch in train_dataloader:
    break
type(batch)

transformers.tokenization_utils_base.BatchEncoding

**Note:** A batch is a `BatchEncoding` object, not a `dict` (even though it looks and works similar to a `dict`).

In [46]:
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 72]),
 'token_type_ids': torch.Size([8, 72]),
 'attention_mask': torch.Size([8, 72])}

In [47]:
from transformers import AutoConfig, AutoModelForSequenceClassification

config = AutoConfig.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)
model

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

**Note:** When we pass a config containing `id2label` and `label2id` to `AutoModelForSequenceClassification.from_pretrained`, we don't need to specify `num_labels`.

In [48]:
outputs = model(**batch)
outputs

SequenceClassifierOutput(loss=tensor(0.6494, grad_fn=<NllLossBackward0>), logits=tensor([[0.2635, 0.3888],
        [0.2568, 0.3924],
        [0.2604, 0.3618],
        [0.2571, 0.3914],
        [0.2683, 0.3804],
        [0.2643, 0.3945],
        [0.2632, 0.3820],
        [0.2542, 0.3819]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

**Note:** `loss` is an attribute of `SequenceClassifierOutput`. If `'labels'` isn't part of the batch, then `loss` will be `None`. This is shown below.

In [49]:
model(**{'input_ids': batch['input_ids'], 'token_type_ids': batch['token_type_ids'], 'attention_mask': batch['attention_mask']})

SequenceClassifierOutput(loss=None, logits=tensor([[0.2635, 0.3888],
        [0.2568, 0.3924],
        [0.2604, 0.3618],
        [0.2571, 0.3914],
        [0.2683, 0.3804],
        [0.2643, 0.3945],
        [0.2632, 0.3820],
        [0.2542, 0.3819]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

It's possible to override the default loss function (`nn.CrossEntropyLoss`) with a different / custom loss function by writing a custom model class. It's also possible to change the amount of dropout before the `classifier` layer. You can refer to Chapter 4 of the Hugging Face book for this.

In [50]:
print(outputs.loss, outputs.logits.shape)

tensor(0.6494, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [51]:
learning_rate = 3e-5

**Note:** The learning rate is significantly smaller than `3e-4` (which works well for ResNet34).

In [52]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=learning_rate)

**Note:** When using linear decay, the value of `lr` specified above will actually be the maximum value. The minimum value will be `0`.

In [53]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
num_training_steps

1377

In [54]:
lr_scheduler_type = "linear"
num_warmup_steps = 0
lr_scheduler = get_scheduler(
    lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)
lr_scheduler

**Note:** `get_scheduler` is a convenience function provided by the Transformers library. But it can be used when training pure PyTorch models as well!

In [55]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [56]:
model.to(device);

Are the parameters of the model's body frozen? Let's check the first layer to find out.

In [57]:
for name, p in model.named_parameters():
    print(type(name))
    print(name)
    print(type(p))
    print(p.shape)
    print(p.requires_grad)
    break

<class 'str'>
bert.embeddings.word_embeddings.weight
<class 'torch.nn.parameter.Parameter'>
torch.Size([30522, 768])
True


No! That means that we're fine-tuning all the layers, not just the randomly instantiated head. (We're also not using discriminative learning rates.)

In [58]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step() # Note: Here, we're updating the learning rate after each step (not after each epoch).
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [59]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch['labels'])

metric.compute()

{'accuracy': 0.8431372549019608, 'f1': 0.8900343642611684}

---

In [60]:
from accelerate import Accelerator

def training_function():
    accelerator = Accelerator()

    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)
    model.to(accelerator.device) # Optional.
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    train_dl, eval_dl, model, optimizer = accelerator.prepare(train_dataloader, eval_dataloader, model, optimizer)

    lr_scheduler = get_scheduler(
        lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )

    progress_bar = tqdm(range(num_training_steps))
    model.train()
    for epoch in range(num_epochs):
        for batch in train_dl:
            batch = {k: v.to(accelerator.device) for k, v in batch.items()} # Optional.
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

    metric = evaluate.load("glue", "mrpc")
    model.eval()
    for batch in eval_dl:
        batch = {k: v.to(accelerator.device) for k, v in batch.items()} # Optional.
        with torch.no_grad():
            outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=accelerator.gather(predictions), references=accelerator.gather(batch['labels']))

    print(metric.compute())

In [61]:
from accelerate import notebook_launcher

notebook_launcher(training_function)

Launching training on one GPU.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1377 [00:00<?, ?it/s]

{'accuracy': 0.8676470588235294, 'f1': 0.9072164948453608}


---

Now, let's write a proper `Engine` class that works with Hugging Face Accelerate.

**Note:** Python always passes references (not values) to the constructor. Only if an argument is immutable (e.g., a string or a tuple), will a modification of the argument within the class lead to a copy being created.

Keeping this in mind, we can pass the training and evaluation datasets to the `Engine` class (without worrying that we'll run out of RAM if multiple instances of the class are created).

In [62]:
from accelerate import Accelerator
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
import wandb
from tqdm.auto import tqdm
import evaluate

class Engine:
    def __init__(self, train_ds, eval_ds, batch_size, data_collator, tokenizer, model, learning_rate, lr_scheduler_type, num_warmup_steps, num_epochs, repo_name):
        self.accelerator = Accelerator()
        self.train_ds = train_ds
        self.eval_ds = eval_ds
        self.train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
        self.eval_dl = DataLoader(eval_ds, batch_size=batch_size * 2, shuffle=False, collate_fn=data_collator)
        self.tokenizer = tokenizer
        self.model = model
        self.model.to(self.accelerator.device) # Optional.
        self.optimizer = AdamW(model.parameters(), lr=learning_rate)
        self.train_dl, self.eval_dl, self.model, self.optimizer = self.accelerator.prepare(self.train_dl, self.eval_dl, self.model, self.optimizer)
        self.num_epochs = num_epochs
        num_training_steps = num_epochs * len(self.train_dl)
        self.lr_scheduler = get_scheduler(
            lr_scheduler_type,
            optimizer=self.optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=num_training_steps
        )
        self.repo_name = repo_name
        wandb_config = {
            'batch_size': batch_size,
            'learning_rate': learning_rate,
            'num_epochs': num_epochs,
            'lr_scheduler_type': lr_scheduler_type,
            'num_warmup_steps': num_warmup_steps
        }
        wandb.init(
            project=repo_name,
            config=wandb_config,
            notes="Logging min of `train_loss` & `eval_loss`, and max of `accuracy` and `F1`."
        )
        wandb.define_metric("train_loss", summary="min")
        wandb.define_metric("eval_loss", summary="min")
        wandb.define_metric("accuracy", summary="max")
        wandb.define_metric("F1", summary="max")

    def train_epoch(self):
        print("Training...")
        self.model.train()
        train_loss = 0
        progress_bar = tqdm(range(len(self.train_dl)))
        for batch in self.train_dl:
            batch = {k: v.to(self.accelerator.device) for k, v in batch.items()} # Optional.
            output = self.model(**batch)
            loss = output.loss
            train_loss += loss.item()
            self.accelerator.backward(loss)
            self.optimizer.step()
            self.lr_scheduler.step()
            self.optimizer.zero_grad()
            progress_bar.update(1)
        train_loss /= len(self.train_ds)
        train_loss = round(train_loss, 4)
        return train_loss

    def evaluate_epoch(self):
        print("Evaluating...")
        self.model.eval()
        eval_loss = 0
        metrics = evaluate.load("glue", "mrpc")
        for batch in self.eval_dl:
            batch = {k: v.to(self.accelerator.device) for k, v in batch.items()} # Optional.
            with torch.no_grad():
                output = self.model(**batch)
            loss = output.loss
            eval_loss += loss.item()
            logits = output.logits
            preds = torch.argmax(logits, dim=-1)
            metrics.add_batch(predictions=self.accelerator.gather(preds), references=self.accelerator.gather(batch['labels']))
        eval_loss /= len(self.eval_ds)
        eval_loss = round(eval_loss, 4)
        results = metrics.compute()
        acc = round(results['accuracy'], 4)
        f1 = round(results['f1'], 4)
        return eval_loss, acc, f1

    def train(self):
        for epoch in range(self.num_epochs):
            print(f"Epoch: {epoch}")

            train_loss = self.train_epoch()
            print(f"Training loss: {train_loss}")

            eval_loss, acc, f1 = self.evaluate_epoch()
            print(f"Evaluation loss: {eval_loss}\nAccuracy: {acc}\nF1: {f1}")

            self.accelerator.wait_for_everyone()
            unwrapped_model = self.accelerator.unwrap_model(self.model)
            if self.accelerator.is_main_process:
                wandb.log({'train_loss': train_loss, 'eval_loss': eval_loss, 'accuracy': acc, 'F1': f1})

                print("Pushing model...")
                unwrapped_model.push_to_hub(self.repo_name, commit_message=f"epoch: {epoch}, accuracy: {acc}, f1: {f1}")
            print("---")
        self.accelerator.wait_for_everyone()
        if self.accelerator.is_main_process:
            wandb.finish()
            print("---")
            print("Pushing tokenizer...")
            self.tokenizer.push_to_hub(self.repo_name, commit_message="pushing tokenizer")
            print("Done!")

In [63]:
from transformers import AutoConfig, AutoModelForSequenceClassification

def training_function():
    checkpoint = "bert-base-uncased"
    config = AutoConfig.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)
    engine = Engine(
        train_ds=tokenized_datasets['train'],
        eval_ds=tokenized_datasets['validation'],
        batch_size=8,
        data_collator=data_collator,
        tokenizer=tokenizer,
        model=model,
        learning_rate=3e-5,
        lr_scheduler_type="linear",
        num_warmup_steps=0,
        num_epochs=3,
        repo_name="bert-base-uncased-finetuned-mrpc-v2"
    )
    engine.train()

In [64]:
from accelerate import notebook_launcher

notebook_launcher(training_function)

Launching training on one GPU.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch: 0
Training...


...

Training loss: 0.0101
Evaluating...
Evaluation loss: 0.0229
Accuracy: 0.875
F1: 0.9116
Pushing model...


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

---


F1,▁██
accuracy,▁██
eval_loss,▁▅█
train_loss,█▄▁


---
Pushing tokenizer...
Done!
